#### Setup

In [1]:
import os, sys
PWD = os.getenv('PWD')
print(PWD)

/protwis/sites/protwis


In [2]:
os.chdir(PWD)
sys.path.insert(0, os.getenv('PWD'))
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "protwis.settings")
import django
django.setup()

/env/lib/python3.4/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


#### Imports

In [3]:
from itertools import combinations
import pickle

from tqdm import tqdm
import pandas as pd


from signprot.interactions import *
from signprot.views import interface_dataset
from protein.models import Protein, ProteinSegment, ProteinFamily
from structure.models import Structure
from signprot.models import SignprotComplex
from seqsign.sequence_signature import SequenceSignature

## Defining the protein sets

### G Proteins

In [4]:
sets = [
    'Gi/o',
    'Gq/11',
    'Gs',
    'G12/13',
]

set_combinations = list(combinations(sets, 2))
segments = list(ProteinSegment.objects.filter(proteinfamily='GPCR'))

In [5]:
set_combinations

[('Gi/o', 'Gq/11'),
 ('Gi/o', 'Gs'),
 ('Gi/o', 'G12/13'),
 ('Gq/11', 'Gs'),
 ('Gq/11', 'G12/13'),
 ('Gs', 'G12/13')]

In [6]:
segments

[<ProteinSegment: N-term>,
 <ProteinSegment: TM1>,
 <ProteinSegment: ICL1>,
 <ProteinSegment: TM2>,
 <ProteinSegment: ECL1>,
 <ProteinSegment: TM3>,
 <ProteinSegment: ICL2>,
 <ProteinSegment: TM4>,
 <ProteinSegment: ECL2>,
 <ProteinSegment: TM5>,
 <ProteinSegment: ICL3>,
 <ProteinSegment: TM6>,
 <ProteinSegment: ECL3>,
 <ProteinSegment: TM7>,
 <ProteinSegment: ICL4>,
 <ProteinSegment: H8>,
 <ProteinSegment: C-term>]

### Querying all Complexes

In [7]:
rec_classes = ProteinFamily.objects.filter(parent=1).exclude(slug=100)

In [8]:
complexes = []
for pc in SignprotComplex.objects.all().prefetch_related('structure', 'protein'):
    t = {}
    
    t['gprot']  = pc.protein.family.name
    t['rec_pdb'] = pc.structure.protein_conformation.protein.entry_short()
    t['rec_obj'] = pc.structure.protein_conformation.protein
    t['rec_class'] = pc.structure.protein_conformation.protein.get_protein_class()
    t['rec_short'] = pc.structure.protein_conformation.protein.parent.entry_short()
    t['rec_name'] = pc.structure.protein_conformation.protein.parent.name
    
    complexes.append(t)

pd.DataFrame(complexes)

,gprot,rec_class,rec_name,rec_obj,rec_pdb,rec_short
0,Gs,Class B1 (Secretin),CT receptor,5uz7,5UZ7,CALCR
1,Gs,Class B1 (Secretin),CT receptor,6niy,6NIY,CALCR
2,Gs,Class B1 (Secretin),GLP-1 receptor,5vai,5VAI,G1SGD4
3,Gs,Class B1 (Secretin),GLP-1 receptor,6b3j,6B3J,GLP1R
4,Gs,Class A (Rhodopsin),A<sub>2A</sub> receptor,6gdg,6GDG,AA2AR
5,Gs,Class A (Rhodopsin),&beta;<sub>2</sub>-adrenoceptor,3sn6,3SN6,ADRB2
6,Gi/o,Class A (Rhodopsin),&mu; receptor,6dde,6DDE,OPRM
7,Gi/o,Class A (Rhodopsin),&mu; receptor,6ddf,6DDF,OPRM
8,Gi/o,Class A (Rhodopsin),Rhodopsin,6cmo,6CMO,OPSD
9,Gi/o,Class A (Rhodopsin),CB<sub>1</sub> receptor,6n4b,6N4B,CNR1


## Calculating a Feature Consensus via the Sequence Signature Tool

In [9]:
def extract_per_attr(data, attr, name):
    '''Return elements of data for which attr is equal to name'''
    return [elem for elem in data if elem[attr] == name]
    

signatures = []
for rec_class in rec_classes:
    class_name = rec_class.name
    data_per_class = None
    data_per_class = extract_per_attr(complexes, 'rec_class', class_name)

    if data_per_class:
        
        for gprot in sets:
            data_per_gprot = None
            data_per_gprot = extract_per_attr(data_per_class, 'gprot', gprot)
            
            if data_per_gprot:
        
                pos_set = [elem['rec_obj'] for elem in data_per_gprot]

                signature = SequenceSignature()
                signature.setup_alignments(segments, pos_set)
                signature.calculate_signature_onesided()

                signatures.append({
                    'rec_class': class_name,
                    'gprot': gprot,
                    'signature': signature,
                })


In [10]:
signatures

[{'gprot': 'Gi/o',
  'rec_class': 'Class A (Rhodopsin)',
  'signature': <seqsign.sequence_signature.SequenceSignature at 0x7f07a325f4e0>},
 {'gprot': 'Gs',
  'rec_class': 'Class A (Rhodopsin)',
  'signature': <seqsign.sequence_signature.SequenceSignature at 0x7f07a2817b00>},
 {'gprot': 'Gs',
  'rec_class': 'Class B1 (Secretin)',
  'signature': <seqsign.sequence_signature.SequenceSignature at 0x7f07a1ed2940>}]

In [11]:
for signature_dict in signatures:
    signature = signature_dict['signature']

    sig_data = signature.prepare_display_data()
    gn = get_generic_numbers(sig_data)
    gn_flat = list(chain.from_iterable(gn))
            
    signature_dict['consensus'] = get_signature_consensus(sig_data, gn_flat)

In [12]:
{'{} - {}'.format(sig['rec_class'], sig['gprot']): len(sig['consensus']) for sig in signatures}

{'Class A (Rhodopsin) - Gi/o': 380,
 'Class A (Rhodopsin) - Gs': 292,
 'Class B1 (Secretin) - Gs': 438}

In [13]:
data = []
for entry in signatures:
    consensus = entry['consensus']
    rec_class = entry['rec_class']
    gprot = entry['gprot']
    while len(consensus) > 0:
        a = consensus.pop()
        a['gprot'] = gprot
        a['rec_class'] = rec_class
        a['origin'] = 'seqsig'
        data.append(a)

df_signatures = pd.DataFrame(data)
df_signatures

,code,cons,feature,gn,gprot,key,length,origin,rec_class,score
0,-,8,Gap (no amino acid),C.01-C-term-0009,Gi/o,379,,seqsig,Class A (Rhodopsin),67
1,-,8,Gap (no amino acid),C.01-C-term-0008,Gi/o,378,,seqsig,Class A (Rhodopsin),67
2,-,8,Gap (no amino acid),C.01-C-term-0007,Gi/o,377,,seqsig,Class A (Rhodopsin),67
3,-,8,Gap (no amino acid),C.01-C-term-0006,Gi/o,376,,seqsig,Class A (Rhodopsin),67
4,-,8,Gap (no amino acid),C.01-C-term-0005,Gi/o,375,,seqsig,Class A (Rhodopsin),67
5,-,8,Gap (no amino acid),C.01-C-term-0004,Gi/o,374,,seqsig,Class A (Rhodopsin),67
6,-,8,Gap (no amino acid),C.01-C-term-0003,Gi/o,373,,seqsig,Class A (Rhodopsin),67
7,-,8,Gap (no amino acid),C.01-C-term-0002,Gi/o,372,,seqsig,Class A (Rhodopsin),67
8,-,7,Gap (no amino acid),C.01-C-term-0001,Gi/o,371,,seqsig,Class A (Rhodopsin),50
9,-,9,Gap (no amino acid),8.60x60,Gi/o,370,,seqsig,Class A (Rhodopsin),83


## Calculating a Feature Consensus via the Interaction Interface Matrix

In [14]:
from pathlib import Path
p = Path('signprot/notebooks/pickles').glob('**/*.p')
files = [x for x in p if x.is_file()]

interface_signatures = []

for file in files:
    if file.is_file():
        with file.open('rb') as f:
            name_raw = str(file)
            a = name_raw.split('/')
            a = a[-1].split('.')
            a = a[0].split('-')
            class_name = a[0].strip()
            gprot = 'Gi/o' if a[1].strip() == 'Gio' else a[1].strip()

            obj = pickle.load(f)            
            interface_signatures.append({
                    'rec_class': class_name,
                    'gprot': gprot,
                    'signature': obj['signature']
                })

In [15]:
interface_signatures

[{'gprot': 'Gs',
  'rec_class': 'Class B1 (Secretin)',
  'signature': <seqsign.sequence_signature.SequenceSignature at 0x7f07a16fcba8>},
 {'gprot': 'Gi/o',
  'rec_class': 'Class A (Rhodopsin)',
  'signature': <seqsign.sequence_signature.SequenceSignature at 0x7f07a1a23d30>},
 {'gprot': 'Gs',
  'rec_class': 'Class A (Rhodopsin)',
  'signature': <seqsign.sequence_signature.SequenceSignature at 0x7f07a19697b8>}]

In [16]:
for signature_dict in interface_signatures:
    signature = signature_dict['signature']

    sig_data = signature.prepare_display_data()
    gn = get_generic_numbers(sig_data)
    gn_flat = list(chain.from_iterable(gn))
            
    signature_dict['consensus'] = get_signature_consensus(sig_data, gn_flat)

In [17]:
{'{} - {}'.format(sig['rec_class'], sig['gprot']): len(sig['consensus']) for sig in interface_signatures}

{'Class A (Rhodopsin) - Gi/o': 36,
 'Class A (Rhodopsin) - Gs': 27,
 'Class B1 (Secretin) - Gs': 21}

In [18]:
data = []
for entry in interface_signatures:
    consensus = entry['consensus']
    rec_class = entry['rec_class']
    gprot = entry['gprot']
    while len(consensus) > 0:
        a = consensus.pop()
        a['gprot'] = gprot
        a['rec_class'] = rec_class
        a['origin'] = 'matrix'
        data.append(a)

df_interface_signatures = pd.DataFrame(data)
df_interface_signatures

,code,cons,feature,gn,gprot,key,length,origin,rec_class,score
0,E,6,Charged negative [E],8.49x49,Gs,20,4,matrix,Class B1 (Secretin),33
1,N,10,Hydrogen bonding [N],8.48x48,Gs,19,,matrix,Class B1 (Secretin),100
2,N,6,Hydrogen bonding [N],8.47x47,Gs,18,,matrix,Class B1 (Secretin),33
3,HY,8,Hydrophobic,7.60x60,Gs,17,any,matrix,Class B1 (Secretin),67
4,E,8,Charged negative [E],6.53x53,Gs,16,4,matrix,Class B1 (Secretin),67
5,G,10,α-Helix flexibility [G],6.50x50,Gs,15,0,matrix,Class B1 (Secretin),100
6,P,10,α-Helix kink [P],6.47x47,Gs,14,2,matrix,Class B1 (Secretin),100
7,HA,10,Hydrophobic aliphatic,6.46x46,Gs,13,2-3,matrix,Class B1 (Secretin),100
8,T,8,Hydrogen bonding [T],6.42x42,Gs,12,,matrix,Class B1 (Secretin),67
9,HA,10,Hydrophobic aliphatic,5.61x61,Gs,11,2-3,matrix,Class B1 (Secretin),100


### Evaluating which consensus features co-occur in both datasets
#### SeqSig and Interaction Interface derived

In [19]:
drop_list = [
    'origin',
    'key',
    'score',
    'cons',
]
v_ds1 = df_signatures.drop(drop_list, 1)
v_ds2 = df_interface_signatures.drop(drop_list, 1)
colnames = v_ds1.columns

ds1 = set([tuple(line) for line in v_ds1.values])
ds2 = set([tuple(line) for line in v_ds2.values])

In [20]:
from IPython.display import display
def summarize_df(df):
    print('Dataframe description:')
    print(df.describe())
    print('\n')
    
    print('Dataframe size:')
    print(df.shape)
    print('\n')
    
    display(df.head())

The Intersection of the sets will show me which consensus features at what position appear in both datasets.

In [21]:
inters = pd.DataFrame(list(ds1.intersection(ds2)))
inters.columns = colnames
summarize_df(inters)

Dataframe description:
       code                feature       gn gprot length            rec_class
count    55                     55       55    55     55                   55
unique   22                     23       43     2     13                    2
top      HA  Hydrophobic aliphatic  5.61x61    Gs    any  Class A (Rhodopsin)
freq     10                     10        3    39     14                   38


Dataframe size:
(55, 6)




,code,feature,gn,gprot,length,rec_class
0,N,Hydrogen bonding [N],8.48x48,Gs,,Class B1 (Secretin)
1,A,Hydrophob al / α-H prop - very high [A],5.64x64,Gs,Max,Class A (Rhodopsin)
2,Hu,Hydrogen bonding uncharged,34.54x54,Gs,3-4,Class A (Rhodopsin)
3,N,Hydrogen bonding [N],8.47x47,Gs,,Class B1 (Secretin)
4,+-,Charged,6.32x32,Gi/o,any,Class A (Rhodopsin)


#### Intersection
- Intersection from SeqSig to Interface
- "Which features from SeqSig also appear in Interface?"
- There are 55 features in common among all tested GPCR and G-Protein combinations.
- Most conserved feature is HA (freq: 10)
- Most conserved position is in TM5 (freq: 3)

In [22]:
differ = pd.DataFrame(list(ds2.difference(ds1)))
differ.columns = colnames
summarize_df(differ)

Dataframe description:
       code               feature       gn gprot length            rec_class
count    29                    29       29    29     29                   29
unique   16                    16       27     2     10                    2
top      HA  Charged positive [R]  5.71x71  Gi/o    any  Class A (Rhodopsin)
freq      3                     3        2    20      5                   25


Dataframe size:
(29, 6)




,code,feature,gn,gprot,length,rec_class
0,Ha,Hydrogen bond acceptor,8.47x47,Gi/o,3,Class A (Rhodopsin)
1,D,Charged negative [D],3.49x49,Gi/o,3,Class A (Rhodopsin)
2,HA,Hydrophobic aliphatic,5.72x72,Gs,3-4,Class A (Rhodopsin)
3,E,Charged negative [E],6.53x53,Gs,4,Class B1 (Secretin)
4,Ha,Hydrogen bond acceptor,2.37x37,Gi/o,2-3,Class A (Rhodopsin)


#### Difference
- Intersection from SeqSig to Interface
- "Which features from Interface can not be found via the SeqSig Tool?"
- 29 features were only found via the interaction interface matrix
- Most frequent feature is HA (freq: 3)
- Most frequent position is in TM5 (freq: 2)

#### Sanity Check
Are there features at position 3.56x56 in the sequence signature dataset?
The previous test says that there are none - lets check that.

In [23]:
df = df_signatures
df.loc[df['gn'] == '3.56x56']

,code,cons,feature,gn,gprot,key,length,origin,rec_class,score
226,+-,9,Charged,3.56x56,Gi/o,153,any,seqsig,Class A (Rhodopsin),83
564,I,7,Hydrophobic aliphatic [I],3.56x56,Gs,107,,seqsig,Class A (Rhodopsin),50
873,T,10,Hydrogen bonding [T],3.56x56,Gs,236,,seqsig,Class B1 (Secretin),100


Yes! Three in total.
How about for the Gi/o g-protein class?

In [24]:
df.loc[
    (df['gn'] == '3.56x56') &
    (df['gprot'] == 'Gi/o')
]

,code,cons,feature,gn,gprot,key,length,origin,rec_class,score
226,+-,9,Charged,3.56x56,Gi/o,153,any,seqsig,Class A (Rhodopsin),83


Only one remaining now – and the feature is not of length 5.

In [25]:
df.loc[
    (df['gn'] == '3.56x56') &
    (df['gprot'] == 'Gi/o') &
    (df['length'] == '5')
]

,code,cons,feature,gn,gprot,key,length,origin,rec_class,score


### How are features represented in different combinations of receptor and g-protein classes?

For this I will use the interaction interface dataset.

In [80]:
from collections import Counter

In [87]:
df = df_interface_signatures
rec_classes = Counter(df['rec_class'].values)
gprot_classes = Counter(df['gprot'].values)

print('Receptor Cl.: {}'.format(rec_classes))
print('G-Prote. Cl.: {} \n'.format(gprot_classes))
rec_classes = sorted(list(rec_classes))
gprot_classes = sorted(list(gprot_classes))

summarize_df(df.drop(drop_list, 1))

Receptor Cl.: Counter({'Class A (Rhodopsin)': 63, 'Class B1 (Secretin)': 21})
G-Prote. Cl.: Counter({'Gs': 48, 'Gi/o': 36}) 

Dataframe description:
       code                feature       gn gprot length            rec_class
count    84                     84       84    84     84                   84
unique   29                     30       57     2     14                    2
top      HA  Hydrophobic aliphatic  8.49x49    Gs    any  Class A (Rhodopsin)
freq     13                     13        3    48     19                   63


Dataframe size:
(84, 6)




,code,feature,gn,gprot,length,rec_class
0,E,Charged negative [E],8.49x49,Gs,4,Class B1 (Secretin)
1,N,Hydrogen bonding [N],8.48x48,Gs,,Class B1 (Secretin)
2,N,Hydrogen bonding [N],8.47x47,Gs,,Class B1 (Secretin)
3,HY,Hydrophobic,7.60x60,Gs,any,Class B1 (Secretin)
4,E,Charged negative [E],6.53x53,Gs,4,Class B1 (Secretin)


#### Class A vs. G-Protein Classes

In [89]:
df1 = df.loc[
    (df['rec_class'] == rec_classes[0]) &
    (df['gprot'] == gprot_classes[0])
]
df2 = df.loc[
    (df['rec_class'] == rec_classes[0]) &
    (df['gprot'] == gprot_classes[1])
]

v_ds1 = df1.drop(drop_list, 1)
v_ds2 = df2.drop(drop_list, 1)
colnames = v_ds1.columns

In [90]:
summarize_df(v_ds1)

Dataframe description:
       code                feature        gn gprot length            rec_class
count    36                     36        36    36     36                   36
unique   15                     15        36     1      9                    1
top      HA  Hydrophobic aliphatic  34.52x52  Gi/o    any  Class A (Rhodopsin)
freq      6                      6         1    36     13                   36


Dataframe size:
(36, 6)




,code,feature,gn,gprot,length,rec_class
21,Hb,Hydrogen bonding (polar),8.49x49,Gi/o,any,Class A (Rhodopsin)
22,+-,Charged,8.48x48,Gi/o,4-5,Class A (Rhodopsin)
23,Ha,Hydrogen bond acceptor,8.47x47,Gi/o,3,Class A (Rhodopsin)
24,αH,α-Helix propensity - high,7.56x56,Gi/o,Hig,Class A (Rhodopsin)
25,L,Hydrophobic aliphatic [L],6.37x37,Gi/o,,Class A (Rhodopsin)


In [91]:
summarize_df(v_ds2)

Dataframe description:
       code               feature       gn gprot length            rec_class
count    27                    27       27    27     27                   27
unique   17                    18       27     1     11                    1
top       R  Charged positive [R]  5.67x67    Gs         Class A (Rhodopsin)
freq      4                     4        1    27      5                   27


Dataframe size:
(27, 6)




,code,feature,gn,gprot,length,rec_class
57,E,Charged negative [E],8.49x49,Gs,4,Class A (Rhodopsin)
58,R,Charged positive [R],8.48x48,Gs,6,Class A (Rhodopsin)
59,R,Charged positive [R],7.56x56,Gs,6,Class A (Rhodopsin)
60,L,Hydrophobic aliphatic [L],6.37x37,Gs,,Class A (Rhodopsin)
61,Hb,Hydrogen bonding,6.36x36,Gs,2,Class A (Rhodopsin)


In [92]:
ds1 = set([tuple(line) for line in v_ds1.values])
ds2 = set([tuple(line) for line in v_ds2.values])

#### Intersection
Which entries do these sets have in common?
In other words: "Which entries are not specific to one receptor + g-protein interaction?"

In [98]:
inter = pd.DataFrame(list(ds2.intersection(ds1)))
try:
    inter.columns = colnames
    inter(differ)
except ValueError as e:
    print('Value Error\n{}:\nNo entries overlap between the two sets.'.format(e))

Value Error
Length mismatch: Expected axis has 0 elements, new values have 6 elements:
No entries overlap between the two sets.


#### Difference
Which entries are unique to each of these sets?
In other words: "Which entries are a unique type of interaction for that recptor + signal protein combination?"
However, if the size of the intersected sets is 0 - then each of the complements must be of the original sets' sizes.